In [2]:
import os
import psycopg2
import google.cloud.bigquery as bigquery
# define global variables to store our DB credentials
PGHOST = 'musicbrainz.postgres.database.azure.com'
PGDATABASE = os.environ.get('PGDATABASE', 'musicbrainz')
PGUSER = os.environ.get('PGUSER', 'musicbrainz')
PGPASSWORD = os.environ.get('PGPASSWORD', 'Git62036242')

# Set the project ID for your BigQuery project.
GBQ_PROJECT_ID = "git62036242-listenbrianz"

# Set the dataset ID for the ListenBrainz dataset.
GBQ_DATASET_ID = "listenbrainz"


sql_beethoven = """
SELECT gid, name, begin_date_year, end_date_year 
  FROM artist 
 WHERE name='Ludwig van Beethoven'
"""

cnx = psycopg2.connect(host=PGHOST,database=PGDATABASE, 
                      user=PGUSER, password=PGPASSWORD)
crs = cnx.cursor()
crs.execute(sql_beethoven)
for result in crs:
    print(result)

# Create a BigQuery client.
client = bigquery.Client(project=GBQ_PROJECT_ID)

# Query the ListenBrainz dataset.


query = """
SELECT artist_name, COUNT(*) as listen_count
FROM `listenbrainz.listenbrainz.listen`
GROUP BY artist_name
ORDER BY listen_count DESC
LIMIT 10;
""".format(GBQ_PROJECT_ID, GBQ_DATASET_ID)

# Execute the query and get the results.
results = client.query(query).result()

# Print the results.
for row in results:
    print(f"{row.artist_name}: {row.listen_count}")


query = """SELECT artist_name, COUNT(DISTINCT user_name) as unique_users_count
FROM  `listenbrainz.listenbrainz.listen`
GROUP BY artist_name
ORDER BY unique_users_count DESC
LIMIT 10""".format(GBQ_PROJECT_ID, GBQ_DATASET_ID)

# Execute the query and get the results.
results = client.query(query).result()

# Print the results.
for row in results:
    print(f"{row.artist_name}: {row.unique_users_count}")


cnx.close()
client.close()

('1f9df192-a621-4f54-8850-2c5373b7eac9', 'Ludwig van Beethoven', 1770, 1827)
Radiohead: 582658
The Beatles: 559065
Pink Floyd: 440631
Daft Punk: 414091
Muse: 335187
Nine Inch Nails: 330685
Aphex Twin: 286798
Coldplay: 283585
Metallica: 278857
Queen: 276178
Radiohead: 1551
Daft Punk: 1506
David Bowie: 1465
Nirvana: 1447
Gorillaz: 1444
Coldplay: 1434
Pink Floyd: 1432
The Beatles: 1406
Red Hot Chili Peppers: 1404
Queen: 1399



Schema for listenbrainz
| Fullname | Mode | Type | Description |
|---|---|---|---|
| listened_at | REQUIRED | TIMESTAMP | Time when the track was listened to |
| user_name | REQUIRED | STRING | Username of the user who listened to the track |
| artist_msid | REQUIRED | STRING | MSID of the artist |
| artist_name | REQUIRED | STRING | Name of the artist |
| artist_mbids | NULLABLE | STRING | MBIDs of the artist |
| release_msid | NULLABLE | STRING | MSID of the release |
| release_name | NULLABLE | STRING | Name of the release |
| release_mbid | NULLABLE | STRING | MBID of the release |
| recording_msid | REQUIRED | STRING | MSID of the recording |
| track_name | REQUIRED | STRING | Name of the track |
| recording_mbid | NULLABLE | STRING | MBID of the recording |
| tags | NULLABLE | STRING | Tags associated with the track |
